In [53]:
import numpy as np
import pandas as pd

In [54]:
# Load the dataset
data = pd.read_csv("Sleep_health_and_lifestyle_dataset.csv")


In [55]:
# Function to make categorical columns into labelled columns
def makelabelled(column):
    second_limit = column.mean()
    first_limit = 0.5 * second_limit
    third_limit = 1.5 * second_limit
    for i in range(0,len(column)):
        if(column[i]<first_limit):
            column[i] = 0
        elif(column[i] < second_limit):
            column[i] = 1
        elif(column[i]<third_limit):
            column[i] = 2
        else:
            column[i] = 3
    return column

In [56]:
# Function to fit the Multinomial Naive Bayes model
def fit(X_train, Y_train):
    result = {}
    class_values = set(Y_train)
    for curr_value in class_values:
        result[curr_value] = {}
        result["total_data"] = len(Y_train)
        curr_class_rows = (Y_train == curr_value)
        X_train_curr = X_train[curr_class_rows]
        Y_train_curr = Y_train[curr_class_rows]
        num_features = X_train.shape[1]
        result[curr_value]["total_count"] = len(Y_train_curr)
        for j in range(1, num_features + 1):
            result[curr_value][j] = {}
            all_possible_values = set(X_train[:, j - 1])
            for this_value in all_possible_values:
                result[curr_value][j][this_value] = (X_train_curr[:, j - 1] == this_value).sum()
    return result

In [57]:
# Function to calculate probability for a single data point
def probability(dictionary, x, current_class):
    output = np.log(dictionary[current_class]["total_count"]) - np.log(dictionary["total_data"])
    num_features = len(dictionary[current_class].keys()) - 1
    for j in range(1, num_features + 1):
        xj = x[j - 1]
        count_current_class_with_value_xj = dictionary[current_class][j][xj] + 1
        count_current_class = dictionary[current_class]["total_count"] + len(dictionary[current_class][j].keys())
        current_xj_prob = np.log(count_current_class_with_value_xj) - np.log(count_current_class)
        output = output + current_xj_prob
    return output

In [58]:
# Function to predict the class for a single data point
def predictSinglePoint(dictionary, x):
    classes = dictionary.keys()
    best_p = -1000
    best_class = -1
    first_run = True
    for current_class in classes:
        if current_class == "total_data":
            continue
        p_curr_class = probability(dictionary, x, current_class)
        if first_run or p_curr_class > best_p:
            best_p = p_curr_class
            best_class = current_class
        first_run = False
    return best_class



In [59]:
# Function to predict classes for multiple data points
def predict(dictionary, X_test):
    Y_pred = []
    for x in X_test:
        x_class = predictSinglePoint(dictionary, x)
        Y_pred.append(x_class)
    return Y_pred

In [60]:
# Convert categorical columns to numerical labels
from sklearn.preprocessing import LabelEncoder
label_encoders = {}
for column in ["Gender", "Occupation", "BMI Category"]:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])


In [61]:
# Convert blood pressure column to numerical labels
data['Blood Pressure'] = data['Blood Pressure'].apply(lambda x: x.split('/')[0] if isinstance(x, str) else x).astype(float)

# Now you can proceed with the label encoding
label_encoders['Blood Pressure'] = LabelEncoder()
data['Blood Pressure'] = label_encoders['Blood Pressure'].fit_transform(data['Blood Pressure'])


In [62]:
# Prepare X and y
X = data.drop(["Person ID", "Quality of Sleep"], axis=1).values
y = data["Quality of Sleep"].values

In [63]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [64]:
# Fit the model
dictionary = fit(X_train, Y_train)

In [65]:
# Function to predict the class for a single data point
def predictSinglePoint(dictionary, x):
    classes = dictionary.keys()
    best_p = -1000
    best_class = -1
    first_run = True
    for current_class in classes:
        if current_class == "total_data":
            continue
        p_curr_class = probability(dictionary, x, current_class)
        print("Current class:", current_class)
        print("Dictionary keys:", dictionary[current_class].keys())
        print("x:", x)
        if first_run or p_curr_class > best_p:
            best_p = p_curr_class
            best_class = current_class
        first_run = False
    return best_class


In [66]:
# Fix the probability function to handle cases where xj might not be present in the dictionary
def probability(dictionary, x, current_class):
    output = np.log(dictionary[current_class]["total_count"]) - np.log(dictionary["total_data"])
    num_features = len(dictionary[current_class].keys()) - 1
    for j in range(1, num_features + 1):
        xj = x[j - 1]
        if xj not in dictionary[current_class][j]:
            # Handle the case where xj is not present in the dictionary
            # You can assign a small value or handle it according to your application
            count_current_class_with_value_xj = 1
        else:
            count_current_class_with_value_xj = dictionary[current_class][j][xj] + 1
        count_current_class = dictionary[current_class]["total_count"] + len(dictionary[current_class][j].keys())
        current_xj_prob = np.log(count_current_class_with_value_xj) - np.log(count_current_class)
        output = output + current_xj_prob
    return output

In [67]:
# Predictions
Y_pred = predict(dictionary, X_test)


Current class: 4
Dictionary keys: dict_keys(['total_count', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
x: [0 37 0 7.2 60 4 0 0 68 7000 nan]
Current class: 5
Dictionary keys: dict_keys(['total_count', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
x: [0 37 0 7.2 60 4 0 0 68 7000 nan]
Current class: 6
Dictionary keys: dict_keys(['total_count', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
x: [0 37 0 7.2 60 4 0 0 68 7000 nan]
Current class: 7
Dictionary keys: dict_keys(['total_count', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
x: [0 37 0 7.2 60 4 0 0 68 7000 nan]
Current class: 8
Dictionary keys: dict_keys(['total_count', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
x: [0 37 0 7.2 60 4 0 0 68 7000 nan]
Current class: 9
Dictionary keys: dict_keys(['total_count', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
x: [0 37 0 7.2 60 4 0 0 68 7000 nan]
Current class: 4
Dictionary keys: dict_keys(['total_count', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
x: [0 50 5 6.0 90 8 3 16 75 10000 'Sleep Apnea']
Current class: 5
Dictionary keys: dict_keys(['total_count

### Implememtation of Multinomial Naive Bayes from Scratch

In [68]:

# Evaluation
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

              precision    recall  f1-score   support

           4       0.33      1.00      0.50         2
           5       0.00      0.00      0.00         0
           6       1.00      0.91      0.96        35
           7       0.95      0.91      0.93        23
           8       1.00      0.89      0.94        19
           9       1.00      0.87      0.93        15

    accuracy                           0.90        94
   macro avg       0.71      0.76      0.71        94
weighted avg       0.97      0.90      0.93        94

[[ 2  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  2 32  0  0  0]
 [ 2  0  0 21  0  0]
 [ 0  1  0  1 17  0]
 [ 1  1  0  0  0 13]]


C:\Users\Nojus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nojus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nojus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetri